In [1]:
import sys

sys.path.insert(0, "../src")

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

nyc_app_token = os.getenv("NYC_OPENDATA_APPTOKEN")

In [3]:
from data_downloader import OpenDataDownloader, GeometryFormatter

loader = OpenDataDownloader(nyc_app_token)

In [4]:
crashes = loader.load_data("crashes", limit=10000)
streets = loader.load_data("centerline")

In [5]:
crashes_gdf = GeometryFormatter(crashes).from_lat_long()
streets_gdf = GeometryFormatter(streets).from_geometry_column("the_geom")

In [29]:
streets_gdf.post_type.unique()

array(['BLVD', 'PL', 'ST', 'TUNL', 'PKWY', 'AVE', nan, 'LN', 'OPAS',
       'ROW', 'ALY', 'PATH', 'DR', 'SQ', 'PLZ', 'PARK', 'SLIP', 'RD',
       'EXPY', 'TPKE', 'EXIT', 'BRG', 'CIR', 'CT', 'PROM', 'CRES', 'FWY',
       'TER', 'LK', 'WAY', 'DY', 'HWY', 'CONC', 'TRWY', 'TRL', 'VLG',
       'OVAL', 'HTS', 'WALK', 'GRN', 'LOOP', 'CP', 'VIA', 'XING', 'CLOS',
       'RAMP', 'EN', 'ESPL', 'GLN', 'HL', 'CV', 'ET', 'MALL', 'MEWS',
       'CRSE', 'CMNS', 'RDG', 'UPAS', 'GDNS', 'MNR', 'ANX', 'EST', 'BDWK',
       'RDWY', 'ROAD', 'DRWY', 'STWY', 'PT', 'RTE', 'BCH', 'BRDG'],
      dtype=object)

In [ ]:
from data_helpers import RoadFeaturesCalculator

collision_rates = RoadFeaturesCalculator(
    features_df=crashes_gdf, streets_df=streets_gdf
).calculate_road_features(
    id_column="collision_id", feature_name="collision_rate", date_column="crash_date"
)

In [ ]:
collision_rates.head()

,physicalid,collision_rate
0,-1,0.0
1,10000,0.0
2,100000,0.0
3,100001,0.0
4,100003,0.0


In [ ]:
trees = loader.load_data("trees")

In [ ]:
trees_gdf = GeometryFormatter(trees).from_geometry_column("the_geom")

In [ ]:
trees_gdf.head()

,created_at,tree_id,block_id,geometry,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,...,st_assem,st_senate,nta,nta_name,boro_ct,state,latitude,longitude,x_sp,y_sp
0,08/27/2015,180683,348711,POINT (1027431.148 202756.769),3,0,OnCurb,Alive,Fair,Acer rubrum,...,28,16,QN17,Forest Hills,4073900,New York,40.72309177,-73.84421522,1027431.14821,202756.768749
1,09/03/2015,200540,315986,POINT (1034455.701 228644.837),21,0,OnCurb,Alive,Fair,Quercus palustris,...,27,11,QN49,Whitestone,4097300,New York,40.79411067,-73.81867946,1034455.70109,228644.837379
2,09/05/2015,204026,218365,POINT (1001822.831 200716.891),3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,...,50,18,BK90,East Williamsburg,3044900,New York,40.71758074,-73.9366077,1001822.83131,200716.891267
3,09/05/2015,204337,217969,POINT (1002420.358 199244.253),10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,...,53,18,BK90,East Williamsburg,3044900,New York,40.71353749,-73.93445616,1002420.35833,199244.253136
4,08/30/2015,189565,223043,POINT (990913.775 182202.426),21,0,OnCurb,Alive,Good,Tilia americana,...,44,21,BK37,Park Slope-Gowanus,3016500,New York,40.66677776,-73.97597938,990913.775046,182202.425999


In [ ]:
trees_per_road = RoadFeaturesCalculator(
    features_df=trees_gdf, streets_df=streets_gdf
).calculate_road_features(
    id_column="tree_id", feature_name="number_of_trees", buffer=40
)

In [ ]:
trees_per_road.number_of_trees.value_counts()

number_of_trees
0.0    121617
1.0        92
2.0         6
4.0         2
3.0         1
Name: count, dtype: int64

In [6]:
parking_meters = loader.load_data("parking_meters", limit=1000)

In [8]:
parking_meters_gdf = GeometryFormatter(parking_meters).from_geometry_column("location")

In [9]:
parking_meters_gdf.head()

,objectid,meter_number,status,pay_by_cell_number,meter_hours,facility,borough,on_street,side_of_street,from_street,...,long,x,y,geometry,:@computed_region_efsh_h5xi,:@computed_region_f5dn_yrer,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih,parking_facility_name
0,7343,2103009,Active,212111,2 HR Pas Mon-Sat 0800-1900,On Street,Bronx,EAST BURNSIDE AVENUE,N,CRESTON AVENUE,...,-73.904419903249,1010691.76403356,250004.230684146,POINT (1010691.764 250004.231),10931,6,5,29,29,NaN
1,15329,3353030,Active,312662,2 HR Pas 0900-1900 Mon-Thur / 4HR Pas 0900-190...,On Street,Brooklyn,PARK AVENUE,N,SPENCER STREET,...,-73.9552816025232,996650.113639221,193211.727799565,POINT (996650.114 193211.728),17212,69,2,38,51,NaN
2,4585,3423040,Active,316276,2 HR Pas Mon-Sat 0800-1900,On Street,Brooklyn,3 AVENUE,W,93 STREET,...,-74.0332943182264,975006.800298631,164514.211703897,POINT (975006.800 164514.212),17216,10,2,44,41,NaN
3,7288,2203017,Active,221923,2 HR Pas Mon-Sat 0830-1900,On Street,Bronx,WESTCHESTER AVENUE,S,ELDER AVENUE,...,-73.8786462367195,1017834.18872505,241155.741163313,POINT (1017834.189 241155.741),11610,58,5,31,26,NaN
4,13346,2083041,Active,209435,2 HR Pas Mon-Sat 0900-1900,On Street,Bronx,EAST 233 STREET,N,WHITE PLAINS ROAD,...,-73.8567493794036,1023855.4709738,264758.892119646,POINT (1023855.471 264758.892),11275,29,5,40,30,NaN


In [7]:
parking_meters.head()

,objectid,meter_number,status,pay_by_cell_number,meter_hours,facility,borough,on_street,side_of_street,from_street,...,long,x,y,location,:@computed_region_efsh_h5xi,:@computed_region_f5dn_yrer,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih,parking_facility_name
0,7343,2103009,Active,212111,2 HR Pas Mon-Sat 0800-1900,On Street,Bronx,EAST BURNSIDE AVENUE,N,CRESTON AVENUE,...,-73.904419903249,1010691.76403356,250004.230684146,"{'type': 'Point', 'coordinates': [-73.90441990...",10931,6,5,29,29,NaN
1,15329,3353030,Active,312662,2 HR Pas 0900-1900 Mon-Thur / 4HR Pas 0900-190...,On Street,Brooklyn,PARK AVENUE,N,SPENCER STREET,...,-73.9552816025232,996650.113639221,193211.727799565,"{'type': 'Point', 'coordinates': [-73.95528160...",17212,69,2,38,51,NaN
2,4585,3423040,Active,316276,2 HR Pas Mon-Sat 0800-1900,On Street,Brooklyn,3 AVENUE,W,93 STREET,...,-74.0332943182264,975006.800298631,164514.211703897,"{'type': 'Point', 'coordinates': [-74.03329431...",17216,10,2,44,41,NaN
3,7288,2203017,Active,221923,2 HR Pas Mon-Sat 0830-1900,On Street,Bronx,WESTCHESTER AVENUE,S,ELDER AVENUE,...,-73.8786462367195,1017834.18872505,241155.741163313,"{'type': 'Point', 'coordinates': [-73.87864623...",11610,58,5,31,26,NaN
4,13346,2083041,Active,209435,2 HR Pas Mon-Sat 0900-1900,On Street,Bronx,EAST 233 STREET,N,WHITE PLAINS ROAD,...,-73.8567493794036,1023855.4709738,264758.892119646,"{'type': 'Point', 'coordinates': [-73.85674937...",11275,29,5,40,30,NaN
